# Notebook to show how an intervened model is calibrated using PyCIEMSS

As in Scenario 1, we want to calibrate a model to data where there should be one transmission rate from `t = 0` to `t = 40`, and another from `t = 40` to `t = 60`

### Load dependencies

In [1]:
import os
import pyciemss
import torch
import pyro
import pyro.distributions as dist

import pyciemss.visuals.plots as plots
import pyciemss.visuals.vega as vega
import pyciemss.visuals.trajectories as trajectories

### Collect model and data

In [2]:
COUNTRY = "France"
MAIN_PATH = "https://raw.githubusercontent.com/ciemss/program-milestones/epi-scenario-1/18-month-milestone/hackathon/epi/scenario1/ciemss/"
model = os.path.join(MAIN_PATH, "SEIIRHD_model_" + COUNTRY + ".json")
dataset = os.path.join(MAIN_PATH, COUNTRY + "_data2.csv")

## Define the intervention as a function or distribution 
Either option will work and allow for calibration before and after the intervention.

In [3]:
static_intervention40A = {torch.tensor(40.0): {"beta_mild": lambda x : x/2, 
                                               "beta_case": lambda x : x/2}}

mask_effect = pyro.sample("mask_effect", dist.Uniform(0,1))
static_intervention40B = {torch.tensor(40.0): {"beta_mild": lambda x: x*mask_effect, 
                                               "beta_case": lambda x: x*mask_effect}}

### Calibrate using `static_intervention40A`

In [4]:
num_iterations = 100
data_mapping = {"cases": "infected"} 
calibrated_results = pyciemss.calibrate(model, dataset, data_mapping=data_mapping, 
                                        static_parameter_interventions=static_intervention40A,
                                        num_iterations=num_iterations)
parameter_estimates = calibrated_results["inferred_parameters"]
parameter_estimates()

/Users/altu809/Projects/pyciemss/pyciemss/integration_utils/observation.py:49: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  if not data_df.applymap(lambda x: isinstance(x, (int, float))).all().all():


Data printout: This dataset contains 58 rows of data. The first column, Timestamp, begins at 1.0 and ends at 59.0. The subsequent columns are named: cases


{'persistent_beta_mild': tensor(0.2415, grad_fn=<ExpandBackward0>),
 'persistent_beta_case': tensor(0.3573, grad_fn=<ExpandBackward0>),
 'persistent_gamma_mildR': tensor(0.1436, grad_fn=<ExpandBackward0>),
 'persistent_gamma_caseH': tensor(0.1341, grad_fn=<ExpandBackward0>),
 'persistent_delta_caseR': tensor(0.2252, grad_fn=<ExpandBackward0>),
 'persistent_mu_caseD': tensor(0.0259, grad_fn=<ExpandBackward0>),
 'persistent_Im0': tensor(1.8376, grad_fn=<ExpandBackward0>),
 'persistent_Ic0': tensor(1.8201, grad_fn=<ExpandBackward0>)}

### Calibrate using `static_intervention40B`

In [6]:
num_iterations = 100
data_mapping = {"cases": "infected"} 
calibrated_results = pyciemss.calibrate(model, dataset, data_mapping=data_mapping, 
                                        static_parameter_interventions=static_intervention40B,
                                        num_iterations=num_iterations)
parameter_estimates = calibrated_results["inferred_parameters"]
parameter_estimates()

/Users/altu809/Projects/pyciemss/pyciemss/integration_utils/observation.py:49: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  if not data_df.applymap(lambda x: isinstance(x, (int, float))).all().all():


Data printout: This dataset contains 58 rows of data. The first column, Timestamp, begins at 1.0 and ends at 59.0. The subsequent columns are named: cases


{'persistent_beta_mild': tensor(0.2165, grad_fn=<ExpandBackward0>),
 'persistent_beta_case': tensor(0.3805, grad_fn=<ExpandBackward0>),
 'persistent_gamma_mildR': tensor(0.1424, grad_fn=<ExpandBackward0>),
 'persistent_gamma_caseH': tensor(0.1413, grad_fn=<ExpandBackward0>),
 'persistent_delta_caseR': tensor(0.2284, grad_fn=<ExpandBackward0>),
 'persistent_mu_caseD': tensor(0.0234, grad_fn=<ExpandBackward0>),
 'persistent_Im0': tensor(1.7697, grad_fn=<ExpandBackward0>),
 'persistent_Ic0': tensor(2.1896, grad_fn=<ExpandBackward0>)}